In [1]:
import pandas as pd
import joblib

In [2]:
df = pd.read_csv("weatherAUS.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
melb_df = df[df["Location"]=="Melbourne"]

In [3]:
melb_df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
67200,2008-07-01,Melbourne,9.5,15.4,2.0,2.8,7.0,W,63.0,W,...,60.0,38.0,1006.8,1010.3,1.0,7.0,11.0,14.6,Yes,No
67201,2008-07-02,Melbourne,11.0,14.5,0.6,5.2,4.0,W,63.0,W,...,66.0,52.0,1013.4,1016.2,7.0,4.0,11.8,13.7,No,Yes
67202,2008-07-03,Melbourne,8.4,14.1,1.4,2.0,0.8,W,39.0,W,...,72.0,58.0,1024.8,1024.3,7.0,7.0,10.6,13.9,Yes,No
67203,2008-07-04,Melbourne,9.1,14.5,0.6,1.8,4.2,E,19.0,NNW,...,87.0,49.0,1030.2,1029.0,7.0,6.0,10.2,13.1,No,No
67204,2008-07-05,Melbourne,4.3,15.5,0.0,0.8,8.4,N,52.0,N,...,81.0,52.0,1028.9,1024.9,2.0,5.0,5.5,14.6,No,No


In [4]:
melb_df["Cloudiness"]=(melb_df["Cloud9am"]+melb_df["Cloud3pm"])/2/8*100
melb_df["Humidity"]=(melb_df["Humidity9am"]+melb_df["Humidity3pm"])/2
melb_df["Pressure"]=(melb_df["Pressure9am"]+melb_df["Pressure3pm"])/2

/opt/anaconda3/envs/MachineL/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/envs/MachineL/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/MachineL/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [5]:
y = melb_df["RainTomorrow"]

In [6]:
clean_melb_df = melb_df
clean_melb_df["Month"]=clean_melb_df.apply(lambda x: int(x["Date"].split("-")[1]),axis=1)
clean_melb_df = melb_df.drop(["RainTomorrow","Date","Location","WindGustDir","Evaporation", "Sunshine","WindDir9am","WindDir3pm", "WindSpeed9am", "WindSpeed3pm", "Humidity9am", "Humidity3pm", "Cloud9am", "Cloud3pm","Pressure9am","Pressure3pm","Temp9am","Temp3pm"],axis=1)

/opt/anaconda3/envs/MachineL/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
clean_melb_df.RainToday = clean_melb_df.RainToday.eq('Yes').mul(1)

In [8]:
X = clean_melb_df
X

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,RainToday,Cloudiness,Humidity,Pressure,Month
67200,9.5,15.4,2.0,63.0,1,50.00,49.0,1008.55,7
67201,11.0,14.5,0.6,63.0,0,68.75,59.0,1014.80,7
67202,8.4,14.1,1.4,39.0,1,87.50,65.0,1024.55,7
67203,9.1,14.5,0.6,19.0,0,81.25,68.0,1029.60,7
67204,4.3,15.5,0.0,52.0,0,43.75,66.5,1026.90,7
...,...,...,...,...,...,...,...,...,...
70388,7.8,13.5,0.6,24.0,0,87.50,90.0,1028.60,6
70389,6.7,14.1,0.2,24.0,0,87.50,74.5,1026.50,6
70390,7.4,14.9,0.0,44.0,0,50.00,70.0,1016.30,6
70391,8.3,12.3,1.4,30.0,1,56.25,69.0,1018.05,6


In [9]:
check = X.groupby(["RainToday"]).count()
check["Percent"]=check["Month"]/1898
check

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,Cloudiness,Humidity,Pressure,Month,Percent
RainToday,,,,,,,,,
0,1385,1385,1385,1385,1385,1385,1385,1385,0.729715
1,513,513,513,513,513,513,513,513,0.270285


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

## Pre-processing

Scale the data using the MinMaxScaler

## Model 1: Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

/opt/anaconda3/envs/MachineL/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [12]:
log_predictions = log_model.predict(X_test)
sample = pd.DataFrame({"Prediction": log_predictions, "Actual": y_test}).reset_index(drop=True)

In [13]:
from sklearn.metrics import f1_score
f1_score(y_test, log_predictions, average=None)

array([0.87287025, 0.48128342])

## Model 2: SVM

In [14]:
from sklearn.svm import SVC 
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [15]:
# Calculate classification report
from sklearn.metrics import classification_report
svm_predictions = svm_model.predict(X_test)

In [16]:
f1_score(y_test, svm_predictions, average=None)

array([0.87220026, 0.4921466 ])

## Model 3: Random Forest

In [17]:
target = y
target_names = ["No", "Yes"]

In [18]:
data = clean_melb_df
feature_names = data.columns

In [19]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)
clf_predictions = clf.predict(X_test)

In [20]:
f1_score(y_test, clf_predictions, average=None)

array([0.83076923, 0.48510638])

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf_predictions=rf.predict(X_test)

In [22]:
f1_score(y_test, rf_predictions, average=None)

array([0.87843137, 0.4972973 ])

In [23]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.21753542227567416, 'Pressure'),
 (0.1667555786836344, 'Humidity'),
 (0.12679368633823854, 'WindGustSpeed'),
 (0.12310851302183383, 'MaxTemp'),
 (0.11514843694542042, 'MinTemp'),
 (0.10956057801047396, 'Cloudiness'),
 (0.06915147903604892, 'Month'),
 (0.06195481202356485, 'Rainfall'),
 (0.009991493665111073, 'RainToday')]

In [24]:
filename = 'finalized_model.sav'
joblib.dump(rf, filename)

['finalized_model.sav']

In [26]:
test = joblib.load(filename)

In [35]:
test.predict([X_test.values[15]])

array(['Yes'], dtype=object)

In [36]:
test_list = [12.7,22,0.8,31,0,68,42,1007,12]
test.predict([test_list])

array(['No'], dtype=object)

In [88]:
import requests

apiKey = "8a1225ee25da9bd6028df45cea70df26"
url = f"https://api.openweathermap.org/data/2.5/onecall?lat=-37.81&lon=144.96&units=metric&exclude=minute,hourly,current,alerts&appid={apiKey}";

response = requests.get(url).json()
TODAYS_DATA = response["daily"][0]
TODAYS_DATA["weather"][0]["main"]


'Clear'

In [59]:
rain = 5
if rain > 1:
    rain_today = 1
else:
    rain_today = 0
rain_today

1

In [97]:
icon_id = TODAYS_DATA["weather"][0]["icon"]
icon = f"http://openweathermap.org/img/wn/{icon_id}@2x.png"

'http://openweathermap.org/img/wn/01d@2x.png'

In [71]:
today = TODAYS_DATA

day = datetime.today()
datem = datetime(day.year, day.month, 1)
month = datem.month

try:
    rain = TODAYS_DATA["rain"]
except:
    rain = 0

if rain > 1:
    rain_today = 1
else:
    rain_today = 0

feature_list = [today["temp"]["min"], today["temp"]["min"], rain, today["wind_speed"], rain_today,today["clouds"], today["humidity"], today["pressure"],month]
feature_list

[14.85, 14.85, 0, 0.71, 0, 22, 51, 1021, 2]

In [73]:
rf.predict([feature_list])

array(['No'], dtype=object)

In [74]:
today = TODAYS_DATA

day = datetime.today()
datem = datetime(day.year, day.month, 1)
month = datem.month

try:
    rain = TODAYS_DATA["rain"]
except:
    rain = 0

if rain > 1:
    rain_today = 1
else:
    rain_today = 0

feature_list = [today["temp"]["min"], today["temp"]["min"], rain, today["wind_speed"], rain_today,today["clouds"], today["humidity"], today["pressure"],month]

k = rf.predict([feature_list])
k

array(['No'], dtype=object)